In [1]:
import sys

sys.path.insert(0, '/home/jianx/search-exposure/')
import faiss
import numpy as np
import forward_ranker.load_data as load_data
from forward_ranker.utils import print_message
from forward_ranker.utils import timestamp
obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer
import random

In [2]:
TRAIN_SIZE = 200_000
TEST_SIZE = 20_000
N_QUERIES = 250_000

all_lcs = set(random.sample(range(8841823), TRAIN_SIZE+TEST_SIZE))
test_lcs = set(random.sample(all_lcs, TEST_SIZE))
train_lcs = all_lcs - test_lcs
print(len(test_lcs),len(train_lcs))
print(test_lcs)

20000 200000
{4849664, 5373954, 4587525, 4718603, 6291496, 131123, 7864371, 1310778, 2228290, 4456516, 6160463, 1966162, 6946903, 6684765, 7340127, 917608, 4718696, 5111928, 6553722, 8388731, 7209104, 3539094, 4980889, 2359459, 3801259, 6946988, 4587693, 6291631, 786610, 393415, 6422735, 655571, 4456661, 7995609, 7995613, 4063457, 4718831, 5505273, 2752765, 1048831, 2752770, 2097419, 4587794, 3670293, 655638, 7995679, 1573151, 1966388, 5112118, 3670337, 4194626, 2490713, 2883938, 5374332, 7078273, 2359687, 4325768, 393620, 6947221, 1442201, 262555, 6947230, 6160802, 393644, 6029760, 3015105, 1704389, 786888, 3277257, 1311182, 7602638, 1966548, 8651223, 3408350, 6554086, 1704452, 6947349, 7471638, 262679, 6291993, 5767712, 6160947, 5505601, 7471682, 5374535, 6685261, 8520285, 7471711, 2228834, 8782447, 4588145, 5505664, 3146369, 787074, 7471751, 6685334, 1311399, 4850358, 8782519, 8127172, 2884302, 1573585, 4588243, 4195041, 6292200, 656110, 7471872, 1966857, 1311507, 1835801, 1835806, 

In [ ]:
INPUT_PATH = "/datadrive/jianx/data/train_data/ance_training_rank100_nqueries250000_8841823.csv"
TRAIN_PATH = "/datadrive/jianx/data/train_data/ance_training_rank100_nqueries{}_{}_random_{}.csv".format(N_QUERIES,TRAIN_SIZE, timestamp())
TEST_PATH = "/datadrive/jianx/data/train_data/ance_testing_rank100_nqueries{}_{}_random_{}.csv".format(N_QUERIES, TEST_SIZE, timestamp())

In [4]:
with open(TRAIN_PATH, "w+") as f:
    f.write("")
with open(TEST_PATH, "w+") as f:
    f.write("")

with open(INPUT_PATH, "r") as f:
    lc = 0
    for line in f:
        if lc//100 in train_lcs:
            pid = line.split(",")[0]
            qid = line.split(",")[1]
            rank = line.split(",")[2]
            with open(TRAIN_PATH, "a") as outf:
                outf.write("{},{},{}".format(pid,qid,rank))
        elif lc//100 in test_lcs:
            pid = line.split(",")[0]
            qid = line.split(",")[1]
            rank = line.split(",")[2]
            with open(TEST_PATH, "a") as outf:
                outf.write("{},{},{}".format(pid,qid,rank))
        lc += 1

In [5]:
print("Finished")

Finished


In [4]:
train_pos = obj_reader("/datadrive/ruohan/data/train_pos_dict.pb")
train_neg = obj_reader("/datadrive/ruohan/data/train_neg_dict.pb")
test_all = obj_reader("/datadrive/ruohan/data/test_rank_dict.pb")
print(len(train_neg.keys()))
print(len(test_all.keys()))

200000
20000


In [5]:
TRAIN_PATH = "/datadrive/jianx/data/train_data/ance_training_rank100_nqueries{}_{}_random_{}.csv".format("full",TRAIN_SIZE, timestamp())
TEST_PATH = "/datadrive/jianx/data/train_data/ance_testing_rank100_nqueries{}_{}_random_{}.csv".format("full", TEST_SIZE, timestamp())
with open(TRAIN_PATH, "w+") as f:
    f.write("")
with open(TEST_PATH, "w+") as f:
    f.write("")
print(TRAIN_PATH,TEST_PATH)

/datadrive/jianx/data/train_data/ance_training_rank100_nqueriesfull_200000_random_Aug_24_02:29:28.csv /datadrive/jianx/data/train_data/ance_testing_rank100_nqueriesfull_20000_random_Aug_24_02:29:28.csv


In [6]:
# RECOVER
with open("/datadrive/jianx/data/train_data/ance_training_rank100_8841823.csv", "r") as f:
    for line in f:
        if int(line.split(",")[0]) in train_neg.keys():
            with open(TRAIN_PATH, "a") as outf:
                outf.write(line)
        if int(line.split(",")[0]) in test_all.keys():
            with open(TEST_PATH, "a") as outf:
                outf.write(line)